In [1]:
import csv
import math
import numpy as np
import datetime 

import obspy
from obspy import UTCDateTime
from obspy.taup import TauPyModel
from obspy.geodetics.base import locations2degrees
from obspy.clients.fdsn import Client

client = Client("IRIS")



In [2]:
def open_lat_long_file(file, lat_long_list):
    with open(file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                print(f'{", ".join(row)}')
                line_count += 1
            elif line_count == 1:
                print(f'Column names are {", ".join(row)}')
                line_count += 1
            else:
                lat_long_list.append([(float(row[0]), float(row[1])), None])
                line_count += 1
        print(f'Processed {line_count} lines.')
    return lat_long_list


def distance_calc(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371  # km

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c

    return d*1000 #To m

def total_distance(lat_long_list):
    total_dis = 0
    for i, lat_long in enumerate(lat_long_list[:-1]):
        total_dis += distance_calc(lat_long[0], lat_long_list[i + 1][0])
    return total_dis



In [3]:
def assign_channel_number(lat_long_list, chan_spac):
    optical_distance = channel_spacing * total_channel
    dis = total_distance(lat_long_list)
    ratio = dis/optical_distance
    lat_long_list[0][1] = 0 #The first lat/long value being the origin
    
    for i, lat_long in enumerate(lat_long_list[:-1]):
        dis = distance_calc(lat_long[0], lat_long_list[i + 1][0])
        curr = lat_long[1]
        channel_between = round((dis/chan_spac) * ratio)
        lat_long_list[i + 1][1] = channel_between + curr
    return lat_long_list

def create_sim_channels(lat_long_list, chan_spac):
    totalDistance = total_distance(lat_long_list)
    new_channels = []
    new_chan = []
    
    for i, lat_long in enumerate(lat_long_list[:-1]):
        current_chan = lat_long
        next_chan = lat_long_list[i + 1]
        current_chan_num = current_chan[-1]
        next_chan_num = next_chan[-1]
        chan_num_list = np.arange(current_chan_num, next_chan_num)
        num = len(chan_num_list)
        if num == 0:
            new_channels.append(next_chan)
        else:
            x_val = [current_chan_num, next_chan_num]
            lat_val = [current_chan[0][-1], next_chan[0][-1]]
            long_val = [current_chan[0][0], next_chan[0][0]]
            new_lats = np.interp(chan_num_list, x_val, lat_val)
            new_longs = np.interp(chan_num_list, x_val, long_val)
            for i in range(len(chan_num_list)):
                new_channels.append([(round(new_longs[i], 6), round(new_lats[i], 6)), current_chan_num + i])
                
    return new_channels

In [4]:
def add_arrival_times(lat_long_list, t, min_mag):
    m = TauPyModel(model = 'ak135')
    time = UTCDateTime(t.strftime("%Y-%m-%dT%H:%M:%S"))
    event_time = obspy.UTCDateTime(time)
    cat = client.get_events(starttime = event_time - 10, endtime = event_time + 10, minmagnitude = 2)
    origin = cat[0].preferred_origin()
    depth = origin.depth
    if depth < 0:
        depth = 1 
    for i, lat_long in enumerate(lat_long_list):
        lat = lat_long_list[i][0][0]
        long = lat_long_list[i][0][1]
        distance = locations2degrees(origin.latitude, origin.longitude, lat, long)
        arrival = m.get_ray_paths(distance_in_degree = distance, 
                         source_depth_in_km = depth / 1000, phase_list=["S"])
        arrival = arrival[0].time
        lat_long.append(round(arrival, 6))
        lat_long_list[i] = lat_long 
    return lat_long_list

In [5]:
channel_spacing = 6.28 #channel spacing in meters
total_channel = 4555

eq_time = datetime.datetime(2022, 5, 25, 10, 38, 7) #min_mag 2
lat_long_list = open_lat_long_file('fiberroute-seadasn.csv', [])

#optical distance number of channels in file vs total distance of cable. 

lat_long_list = assign_channel_number(lat_long_list, channel_spacing);

Fiber Route
Column names are Lat, Long, Elev, Description
Processed 213 lines.


In [6]:
eq_time = datetime.datetime(2022, 5, 25, 10, 38, 7)
lat_long_list = add_arrival_times(lat_long_list, eq_time, 2)


In [7]:

new_channels_list = create_sim_channels(lat_long_list, channel_spacing)
new_channels_list = add_arrival_times(new_channels_list, eq_time, 2)